# Aiogram

- Это фреймворк для написания ботов Telegram, с использованием асинхронной парадигмы программирования.
  
Оф документация: https://docs.aiogram.dev/en/latest/

# Функция для запуска бота Telegram

In [ ]:
import asyncio

from aiogram import Bot, Dispatcher
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.fsm.storage.base import BaseStorage
# from aiogram.fsm.storage.redis import RedisStorage

from settings.config import settings


async def bot_startup():
    bot = Bot(
        token=settings.TG_API,
        default=DefaultBotProperties(parse_mode=ParseMode.MARKDOWN_V2),
    )
    
    storage = BaseStorage()# RedisStorage.from_url('redis://localhost:6379/0')
    dp = Dispatcher(storage=storage)
    
    # await dp.include_routers(...)
    await bot.delete_webhook(drop_panding_updates =True)
    await dp.start_polling(bot)
    
    


if __name__ == "__main__":
    asyncio.run(bot_startup())

Рассмотрим подробнее:
- bot -- переменная самого бота, то есть именно этот объект получает запросы от конечного пользователя.
- dp (Dispatcher) -- Диспатчер, своего рода диспечер для бота, он обрабатывает поступающие запросы фильтруя и передавая их в выполнения той части программы которая соответствует этому запросу.

Параметры бота:
- token=settings.TG_API - API-токен для подключения к конкретному боту
- default=DefaultBotProperties(parse_mode=ParseMode.MARKDOWN_V2) - Стандартные параметры бота (каким способом он будет форматировать вывод сообщений)

Параметры диспетчера:
- storage - это хранилище где диспатчер хранит информацию о пользователях которые с ним взаимодействуют

Методы: 
- delete_webhook - Удаляет вебхуки созданные для бота тогда когда он был выключен, или неактивен
- start_polling - основная функция которая активирует прослушивание Диспатчером нашего Бота на наличие сообщений
 

# Наш первый обработчик(хэндлер) сообщений

In [ ]:
from aiogram import Router
from aiogram.types import Message

r = Router()

@r.message()
async def start_message(message: Message):
    print(message)
    print(message.text)
    await message.answer('Привет! Чем могу помочь?')

- В начале каждого файла хэнделера мы указываем Router() объект, именно к нему и будет "цепляться" каждая функция которую мы будем декорировать через Router-объект(в нашем случае это `r`)
- r.message - это хэндлер для каждого сообщения, с любым содержанием. Это так, потому что внутри message() нет никаких фильтров
- в аргументы асинх. функции которую мы пишем и которая декорируется Router'ом, мы обязательно передаем параметры маркированные типом aiogram'a или нашими встроенными типами(но об этом позднее)

Ответы со стороны бота:

Бот может отвечать различными структрами, он может отвечать, как картинками, так и текстом, так и видео, так и документами, то есть любой структурой с которой мы можем взаимодествовать и в обычном telegram. Единственное допущения aiograma это работа с альбомами (т.е. когда идет несколько фото в одном сообщении)

- message.answer --выдает ответ пользователю на его сообщение. Сам message (класса aiogram.types.Message) является объектом-информацией о сообщении пользователя